##### Copyright 2020 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Point Clouds for tensorflow_graphics
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/schellmi42/graphics/blob/point_convolutions/tensorflow_graphics/projects/point_convolutions/pylib/notebooks/Introduction.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/schellmi42/graphics/blob/point_convolutions/tensorflow_graphics/projects/point_convolutions/pylib/notebooks/Introduction.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Initialization


### Clone repositories, and install requirements and custom_op package

In [ ]:
# Clone repositories
!rm -r graphics
!git clone https://github.com/schellmi42/graphics

# install requirements and load tfg module 
!pip install -r graphics/requirements.txt

# install custom ops
!pip install graphics/tensorflow_graphics/projects/point_convolutions/custom_ops/pkg_builds/tf_2.2.0/*.whl


### Load modules

In [ ]:
import sys
# (this is equivalent to export PYTHONPATH='$HOME/graphics:/content/graphics:$PYTHONPATH', but adds path to running session)
sys.path.append("/content/graphics")

# load point cloud module 
# (this is equivalent to export PYTHONPATH='/content/graphics/tensorflow_graphics/projects/point_convolutions:$PYTHONPATH', but adds path to running session)
sys.path.append("/content/graphics/tensorflow_graphics/projects/point_convolutions")

Check if it loads without errors

In [ ]:
import tensorflow as tf
import tensorflow_graphics as tfg
import pylib.pc as pc
import numpy as np

print('TensorFlow version: %s'%tf.__version__)
print('TensorFlow-Graphics version: %s'%tfg.__version__)
print('Point Cloud Module: ', pc)

## Example Code


### 2D square point clouds using segmentation IDs
Here we create a batch of point clouds with variable number of points per cloud from unordered points with an additional id tensor.

The `batch_ids` are the segmentation ids, which indicate which point belongs to which point cloud in the batch. For more information on segmentation IDs see: [tf.math#segmentation](https://www.tensorflow.org/api_docs/python/tf/math#Segmentation)

If the points are ordered by batch id, it is also possible to pass a `sizes` tensor, which has the size of each point cloud in it.

In [ ]:
import numpy as np


def square(num_samples, size=1):
  # 2D square in 3D for easier visualization
  points = np.random.rand(num_samples, 2)*2-1
  return points*size

num_samples=1000
batch_size = 10

# create numpy input data consisting of points and segmentation identifiers
points = square(num_samples)
batch_ids = np.random.randint(0, batch_size, num_samples)

# create tensorflow point cloud
point_cloud = pc.PointCloud(points, batch_ids, batch_size)

# print information
sizes = point_cloud.get_sizes()
print('%s point clouds of sizes:'%point_cloud._batch_size)
print(sizes.numpy())

Create a batch of point hierarchies using sequential poisson disk sampling with pooling radii 0.1, 0.4, 2.

In [ ]:
# numpy input parameters
sampling_radii = np.array([[0.1], [0.4], [2]])

# create tensorflow point hierarchy
point_hierarchy = pc.PointHierarchy(point_cloud,
                                    sampling_radii,
                                    'poisson_disk')

In [ ]:
# print information
num_levels = len(sampling_radii) + 1
print('%s point clouds of sizes:'%point_cloud._batch_size)
sizes = point_hierarchy.get_sizes()
for i in range(num_levels):
  print('level: ' + str(i))
  print(sizes[i].numpy())

assign a shape to the batch and look at the sizes again

In [ ]:
point_hierarchy.set_batch_shape([2, 5])
print('%s point clouds of sizes:'%point_cloud._batch_size)
sizes = point_hierarchy.get_sizes()
for i in range(num_levels):
  print('level: ' + str(i))
  print(sizes[i].numpy())

Visualize the levels of one example from the batch.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# which example from the batch to choose, can be 'int' or relative in [A1,...,An]
batch_id = [0,1]

curr_points = point_hierarchy.get_points(batch_id)

# plotting
plt.figure(figsize=[num_levels*5,5])
for i in range(num_levels):
  plt.subplot(1,num_levels,i+1)
  plt.plot(curr_points[i][:, 0],curr_points[i][:, 1],'bo')
  plt.axis([-1, 1, -1, 1])
  if i==0:
    plt.title('input point cloud')
  else:
    plt.title('poisson sampled points with radius %s'%sampling_radii[i - 1, 0])
    


### 3D point clouds from input files using arbitrary batch sizes with padding

Here we create point clouds from input files using a zero padded representation of shape `[A1, .., An, V, D]`.
Internally this is converted to a segmented representation.



 #### Loading from ASCII .txt files

In [ ]:
import pylib.io as io

# SHREC15

#### get files ####
input_dir = 'graphics/tensorflow_graphics/projects/point_convolutions/test_point_clouds/SHREC15/'
filenames = tf.io.gfile.listdir(input_dir)
batch_size = len(filenames)
print('### batch size ###'); print(batch_size)

for i in range(batch_size):
  filenames[i] = input_dir + filenames[i]

#### load points #####
batch_shape = [5,2]
print('### batch shape###'); print(batch_shape)
points, normals, sizes = io.load_batch_of_points(filenames, batch_shape = batch_shape)

print('### data shape ###'); print(points.shape)
print('### points per point cloud ###');print(sizes.numpy())

#### build point hierarchy #####
point_cloud = pc.PointCloud(points, sizes=sizes)

point_hierarchy = pc.PointHierarchy(point_cloud,
                                    [[0.05], [0.1]],
                                    'poisson_disk')

sizes = point_hierarchy.get_sizes()

print('### point per point cloud in hierarchy ###')
for level in range(len(sizes)):
  print('level %s'%level)
  print(sizes[level].numpy())

### extract points from last level in original batch shape ###
hierarchical_points = point_hierarchy.get_points()
out_points = hierarchical_points[-1]
print('### shape of points in last level ###'); print(out_points.shape)

#### Loading vertices from mesh files 


In [ ]:
# Thingi10k meshes

#### get files ####
input_dir = 'graphics/tensorflow_graphics/projects/point_convolutions/test_point_clouds/meshes/'
filenames = tf.io.gfile.listdir(input_dir)
batch_size = len(filenames)
print('### batch size ###'); print(batch_size)

for i in range(batch_size):
  filenames[i] = input_dir+filenames[i]

#### load points ####
points, sizes = io.load_batch_of_meshes(filenames)

print('### data shape ###'); print(points.shape)
print('### points per point cloud ###');print(sizes.numpy())

#### build a point cloud object ####
point_cloud = pc.PointCloud(points, sizes=sizes)

print('### internal shape conversion ###')
print('Input    (padded):    %s elements'%len(tf.reshape(points, [-1, 3])))
print('Internal (segmented): %s elements'%len(point_cloud._points))

point_hierarchy = pc.PointHierarchy(point_cloud,
                                    [[0.05], [0.1]],
                                    'poisson_disk')

sizes = point_hierarchy.get_sizes()

print('### point per point cloud in hierarchy ###')
for level in range(len(sizes)):
  print('level %s'%level)
  print(sizes[level].numpy())

### Monte-Carlo Convolutions


Create convolutions for a point hierarchy with MLPs as kernel 



In [ ]:
import numpy as np
### create random input data
num_pts = 1000
point_dim = 3
feature_dim = 3
batch_size = 10

# create random points
points = np.random.rand(num_pts,point_dim)
batch_ids = np.random.randint(0,batch_size,num_pts)
batch_ids[:batch_size] = np.arange(0,batch_size) # ensure non-empty point clouds
# create random features
features = np.random.rand(num_pts,feature_dim)

# build initial point cloud
point_cloud = pc.PointCloud(points, batch_ids, batch_size)

# build point hierarchy
sample_radii = np.array([[0.1],[0.2],[2]])
point_hierarchy = pc.PointHierarchy(point_cloud,sample_radii)

### build model

# layer parameters
conv_radii = 2*sample_radii
feature_sizes = [8,16,32]
kernel_hidden_size = 8 # number of neurons in the hidden layer of the kernel MLP

### initialize layers
Conv1 = pc.layers.MCConv(feature_dim, feature_sizes[0], point_dim,kernel_hidden_size)
Conv2 = pc.layers.MCConv(feature_sizes[0],feature_sizes[1],point_dim,kernel_hidden_size)
Conv3 = pc.layers.MCConv(feature_sizes[1],feature_sizes[2],point_dim,kernel_hidden_size)

### call layers
conv1_result = Conv1(features,point_hierarchy[0], point_hierarchy[1],conv_radii[0])
conv2_result = Conv2(conv1_result,point_hierarchy[1], point_hierarchy[2],conv_radii[1])
conv3_result = Conv3(conv2_result,point_hierarchy[2], point_hierarchy[3],conv_radii[2], return_sorted=True)

### printing 
print('### point cloud sizes ###')
sizes = point_hierarchy.get_sizes()
for s in sizes:
  print(s.numpy())

print('\n### features dimensions flat ###')
print('Input: ');print(features.shape)
print('Conv1: ');print(conv1_result.shape)
print('Conv2: ');print(conv2_result.shape)
print('Conv3: ');print(conv3_result.shape)

# again in padded format
point_hierarchy.set_batch_shape([5,2])

unflatten = point_hierarchy[0].get_unflatten()
features_padded = unflatten(features)
### call layers
conv1_result_padded = Conv1(features_padded, point_hierarchy[0], point_hierarchy[1],conv_radii[0], return_padded=True)
conv2_result_padded = Conv2(conv1_result_padded, point_hierarchy[1], point_hierarchy[2],conv_radii[1], return_padded=True)
conv3_result_padded = Conv3(conv2_result_padded, point_hierarchy[2], point_hierarchy[3],conv_radii[2], return_padded=True)
print('\n### feature dimensions padded ###')
print('Input: ');print(features_padded.shape)
print('Conv1: ');print(conv1_result_padded.shape)
print('Conv2: ');print(conv2_result_padded.shape)
print('Conv3: ');print(conv3_result_padded.shape)